In [1]:
import csv
import pandas as pd

Data source:
- Bureau of Reclamation: Lake Mead, Lake Mohave, Lake Havasu
- United States Geological Survey: San Carlos 
- US Army Corps of Engineers (only go to previous 180 days): Alamo, Painted Rock 


In [ ]:
LOWER_RESERVOIR_NAME = {921:'lake mead', 922: 'lake mohave', 923: 'lake havasu', }